In [91]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from timeit import default_timer as timer


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [92]:
#read in train_df
train_df = pd.read_csv('data/Train.csv')

train_df.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12.55.00,ATA,TU 32AIMN,260.0
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16.55.00,ATA,TU 31BIMO,20.0
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06.45.00,ATA,TU 32AIMN,0.0
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17.00.00,ATA,TU 736IOK,0.0
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15.50.00,ATA,TU 320IMU,22.0


In [93]:
# read in test_df
test_df = pd.read_csv('data/Test.csv')

test_df.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC
0,test_id_0,2016-05-04,TU 0700,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07.30.00,ATA,TU 32AIMF
1,test_id_1,2016-05-05,TU 0395,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20.05.00,ATA,TU 320IMW
2,test_id_2,2016-05-06,TU 0745,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12.25.00,ATA,TU 32AIMC
3,test_id_3,2016-05-11,TU 0848,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13.10.00,ATA,TU 31BIMO
4,test_id_4,2016-05-11,TU 0635,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12.35.00,ATA,TU 736IOQ


++ CLEANING TEST_DF ++

In [94]:
test_df = test_df.rename(columns={'DATOP': 'flight_date', 'FLTID': 'flight_no', 'DEPSTN': 'dep_point', 'ARRSTN': 'arr_point', 'STD': 'scheduled_dep', 'STA': 'scheduled_arr', 'STATUS': 'status', 'AC': 'aircraft_code', 'target': 'target'})

print(test_df)

                ID flight_date  flight_no dep_point arr_point   
0        test_id_0  2016-05-04   TU 0700        DJE       TUN  \
1        test_id_1  2016-05-05   TU 0395        TUN       BKO   
2        test_id_2  2016-05-06   TU 0745        FRA       TUN   
3        test_id_3  2016-05-11   TU 0848        BEY       TUN   
4        test_id_4  2016-05-11   TU 0635        ORY       MIR   
...            ...         ...        ...       ...       ...   
9328  test_id_9328  2018-09-12   TU 0998        TUN       NCE   
9329  test_id_9329  2018-09-27  WKL 0000        TUN       TUN   
9330  test_id_9330  2018-09-03   UG 1601        SJJ       TUN   
9331  test_id_9331  2018-09-15   UG 0008        TUN       DJE   
9332  test_id_9332  2018-09-19   UG 0009        DJE       TUN   

            scheduled_dep        scheduled_arr status aircraft_code  
0     2016-05-04 06:40:00  2016-05-04 07.30.00    ATA     TU 32AIMF  
1     2016-05-05 15:20:00  2016-05-05 20.05.00    ATA     TU 320IMW  
2     201

In [95]:
test_df.head()

,ID,flight_date,flight_no,dep_point,arr_point,scheduled_dep,scheduled_arr,status,aircraft_code
0,test_id_0,2016-05-04,TU 0700,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07.30.00,ATA,TU 32AIMF
1,test_id_1,2016-05-05,TU 0395,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20.05.00,ATA,TU 320IMW
2,test_id_2,2016-05-06,TU 0745,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12.25.00,ATA,TU 32AIMC
3,test_id_3,2016-05-11,TU 0848,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13.10.00,ATA,TU 31BIMO
4,test_id_4,2016-05-11,TU 0635,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12.35.00,ATA,TU 736IOQ


In [96]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9333 entries, 0 to 9332
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             9333 non-null   object
 1   flight_date    9333 non-null   object
 2   flight_no      9333 non-null   object
 3   dep_point      9333 non-null   object
 4   arr_point      9333 non-null   object
 5   scheduled_dep  9333 non-null   object
 6   scheduled_arr  9333 non-null   object
 7   status         9333 non-null   object
 8   aircraft_code  9333 non-null   object
dtypes: object(9)
memory usage: 656.4+ KB


In [97]:
test_df['flight_date'] = pd.to_datetime(test_df['flight_date'], format='%Y-%m-%d')
test_df['scheduled_dep'] = pd.to_datetime(test_df['scheduled_dep'], errors='coerce')

In [98]:
# Replace '.' with ':' in 'scheduled_arr' column to correct time format
test_df['scheduled_arr'] = test_df['scheduled_arr'].str.replace('.', ':', regex=False)

# Convert to datetime after formatting correction
test_df['scheduled_arr'] = pd.to_datetime(test_df['scheduled_arr'], errors='coerce')

# Check the result
print(test_df['scheduled_arr'].head())


0   2016-05-04 07:30:00
1   2016-05-05 20:05:00
2   2016-05-06 12:25:00
3   2016-05-11 13:10:00
4   2016-05-11 12:35:00
Name: scheduled_arr, dtype: datetime64[ns]


In [99]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9333 entries, 0 to 9332
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             9333 non-null   object        
 1   flight_date    9333 non-null   datetime64[ns]
 2   flight_no      9333 non-null   object        
 3   dep_point      9333 non-null   object        
 4   arr_point      9333 non-null   object        
 5   scheduled_dep  9333 non-null   datetime64[ns]
 6   scheduled_arr  9333 non-null   datetime64[ns]
 7   status         9333 non-null   object        
 8   aircraft_code  9333 non-null   object        
dtypes: datetime64[ns](3), object(6)
memory usage: 656.4+ KB


In [100]:
test_df.head()

,ID,flight_date,flight_no,dep_point,arr_point,scheduled_dep,scheduled_arr,status,aircraft_code
0,test_id_0,2016-05-04,TU 0700,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07:30:00,ATA,TU 32AIMF
1,test_id_1,2016-05-05,TU 0395,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20:05:00,ATA,TU 320IMW
2,test_id_2,2016-05-06,TU 0745,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12:25:00,ATA,TU 32AIMC
3,test_id_3,2016-05-11,TU 0848,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13:10:00,ATA,TU 31BIMO
4,test_id_4,2016-05-11,TU 0635,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12:35:00,ATA,TU 736IOQ


++ CLEANING TRAIN_DF ++

In [101]:
train_df = train_df.rename(columns={'DATOP': 'flight_date', 'FLTID': 'flight_no', 'DEPSTN': 'dep_point', 'ARRSTN': 'arr_point', 'STD': 'scheduled_dep', 'STA': 'scheduled_arr', 'STATUS': 'status', 'AC': 'aircraft_code', 'target': 'target'})

print(test_df)

                ID flight_date  flight_no dep_point arr_point   
0        test_id_0  2016-05-04   TU 0700        DJE       TUN  \
1        test_id_1  2016-05-05   TU 0395        TUN       BKO   
2        test_id_2  2016-05-06   TU 0745        FRA       TUN   
3        test_id_3  2016-05-11   TU 0848        BEY       TUN   
4        test_id_4  2016-05-11   TU 0635        ORY       MIR   
...            ...         ...        ...       ...       ...   
9328  test_id_9328  2018-09-12   TU 0998        TUN       NCE   
9329  test_id_9329  2018-09-27  WKL 0000        TUN       TUN   
9330  test_id_9330  2018-09-03   UG 1601        SJJ       TUN   
9331  test_id_9331  2018-09-15   UG 0008        TUN       DJE   
9332  test_id_9332  2018-09-19   UG 0009        DJE       TUN   

           scheduled_dep       scheduled_arr status aircraft_code  
0    2016-05-04 06:40:00 2016-05-04 07:30:00    ATA     TU 32AIMF  
1    2016-05-05 15:20:00 2016-05-05 20:05:00    ATA     TU 320IMW  
2    2016-05-06

In [102]:
train_df.head()

,ID,flight_date,flight_no,dep_point,arr_point,scheduled_dep,scheduled_arr,status,aircraft_code,target
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12.55.00,ATA,TU 32AIMN,260.0
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16.55.00,ATA,TU 31BIMO,20.0
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06.45.00,ATA,TU 32AIMN,0.0
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17.00.00,ATA,TU 736IOK,0.0
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15.50.00,ATA,TU 320IMU,22.0


In [103]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107833 entries, 0 to 107832
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             107833 non-null  object 
 1   flight_date    107833 non-null  object 
 2   flight_no      107833 non-null  object 
 3   dep_point      107833 non-null  object 
 4   arr_point      107833 non-null  object 
 5   scheduled_dep  107833 non-null  object 
 6   scheduled_arr  107833 non-null  object 
 7   status         107833 non-null  object 
 8   aircraft_code  107833 non-null  object 
 9   target         107833 non-null  float64
dtypes: float64(1), object(9)
memory usage: 8.2+ MB


In [104]:
# convert to datetime
train_df['flight_date'] = pd.to_datetime(train_df['flight_date'], format='%Y-%m-%d')
train_df['scheduled_dep'] = pd.to_datetime(train_df['scheduled_dep'], errors='coerce')

In [105]:
# Replace '.' with ':' in 'scheduled_arr' column to correct time format
train_df['scheduled_arr'] = train_df['scheduled_arr'].str.replace('.', ':', regex=False)

# Convert to datetime after formatting correction
train_df['scheduled_arr'] = pd.to_datetime(train_df['scheduled_arr'], errors='coerce')

# Check the result
print(train_df['scheduled_arr'].head())

0   2016-01-03 12:55:00
1   2016-01-13 16:55:00
2   2016-01-16 06:45:00
3   2016-01-17 17:00:00
4   2016-01-17 15:50:00
Name: scheduled_arr, dtype: datetime64[ns]


In [114]:
train_df.head()

,ID,flight_date,flight_no,dep_point,arr_point,scheduled_dep,scheduled_arr,status,aircraft_code,target_time
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,ATA,TU 32AIMN,0 days 04:20:00
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,ATA,TU 31BIMO,0 days 00:20:00
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,0 days 00:00:00
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,0 days 00:00:00
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,ATA,TU 320IMU,0 days 00:22:00


In [106]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107833 entries, 0 to 107832
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             107833 non-null  object        
 1   flight_date    107833 non-null  datetime64[ns]
 2   flight_no      107833 non-null  object        
 3   dep_point      107833 non-null  object        
 4   arr_point      107833 non-null  object        
 5   scheduled_dep  107833 non-null  datetime64[ns]
 6   scheduled_arr  107833 non-null  datetime64[ns]
 7   status         107833 non-null  object        
 8   aircraft_code  107833 non-null  object        
 9   target         107833 non-null  float64       
dtypes: datetime64[ns](3), float64(1), object(6)
memory usage: 8.2+ MB


In [107]:
train_df['target']

0         260.0
1          20.0
2           0.0
3           0.0
4          22.0
          ...  
107828      0.0
107829      0.0
107830      0.0
107831      0.0
107832      0.0
Name: target, Length: 107833, dtype: float64

In [108]:
# Angenommen, test_df['target'] enthält die Anzahl der Minuten als float
train_df['target_time'] = pd.to_timedelta(train_df['target'], unit='m')

In [109]:
train_df['target_time']

0        0 days 04:20:00
1        0 days 00:20:00
2        0 days 00:00:00
3        0 days 00:00:00
4        0 days 00:22:00
               ...      
107828   0 days 00:00:00
107829   0 days 00:00:00
107830   0 days 00:00:00
107831   0 days 00:00:00
107832   0 days 00:00:00
Name: target_time, Length: 107833, dtype: timedelta64[ns]

In [111]:
# drop column 'target'
train_df = train_df.drop(columns=['target'])

In [112]:
# Filtere nur die Zeilen heraus, bei denen die `days` Komponente größer als 0 ist
greater_than_zero_days = train_df[train_df['target_time'].dt.days > 0]

# Überprüfen, ob solche Zeilen existieren und ausgeben
if not greater_than_zero_days.empty:
    print("Es gibt Zeilen, bei denen die Tage-Komponente größer als 0 ist:")
    print(greater_than_zero_days)
else:
    print("Es gibt keine Zeilen mit einer Tage-Komponente größer als 0.")


Es gibt Zeilen, bei denen die Tage-Komponente größer als 0 ist:
                     ID flight_date flight_no dep_point arr_point   
1026      train_id_1026  2016-04-15  TU 3441        ORY       MIR  \
2171      train_id_2171  2016-08-20  TU 0268        TUN       SVO   
5134      train_id_5134  2016-06-28  TU 2171        IST       TUN   
10123    train_id_10123  2016-03-20  TU 0209        NTE       TUN   
11293    train_id_11293  2016-08-21  TU 0269        SVO       NBE   
...                 ...         ...       ...       ...       ...   
102074  train_id_102074  2018-11-23  TU 0735        DJE       MIR   
102118  train_id_102118  2018-11-23  TU 0965        MIR       TOE   
102606  train_id_102606  2018-10-29  TU 0964        MIR       LYS   
103026  train_id_103026  2018-12-09  TU 0397        ABJ       OUA   
103027  train_id_103027  2018-12-10  TU 0397        OUA       TUN   

             scheduled_dep       scheduled_arr status aircraft_code   
1026   2016-04-15 14:20:00 2016-04-1

In [113]:
train_df.head()

,ID,flight_date,flight_no,dep_point,arr_point,scheduled_dep,scheduled_arr,status,aircraft_code,target_time
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,ATA,TU 32AIMN,0 days 04:20:00
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,ATA,TU 31BIMO,0 days 00:20:00
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,0 days 00:00:00
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,0 days 00:00:00
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,ATA,TU 320IMU,0 days 00:22:00


In [115]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107833 entries, 0 to 107832
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype          
---  ------         --------------   -----          
 0   ID             107833 non-null  object         
 1   flight_date    107833 non-null  datetime64[ns] 
 2   flight_no      107833 non-null  object         
 3   dep_point      107833 non-null  object         
 4   arr_point      107833 non-null  object         
 5   scheduled_dep  107833 non-null  datetime64[ns] 
 6   scheduled_arr  107833 non-null  datetime64[ns] 
 7   status         107833 non-null  object         
 8   aircraft_code  107833 non-null  object         
 9   target_time    107833 non-null  timedelta64[ns]
dtypes: datetime64[ns](3), object(6), timedelta64[ns](1)
memory usage: 8.2+ MB
